But du notebook : recuperer et organiser les données afin de créer une carte interactive


# 1.Data loading


The data can be downloaded from the bixi montréal website.  
 https://bixi.com/fr/donnees-ouvertes  
you can download them by doing : 
 ```
 wget https://sitewebbixi.s3.amazonaws.com/uploads/docs/2021-stations-87d92f.zip 
 ```


In [ ]:
import pandas as pd 
from datetime import datetime
import os

In [ ]:
# deplacement = pd.read_csv('../transfo-data/2021-rawdata.csv')
deplacement = pd.read_csv('../data/2021_donnees_ouvertes.csv')
deplacement.head()

In [ ]:
deplacement = deplacement.drop(columns=["end_date","duration_sec","is_member"])
deplacement_unique = deplacement.groupby(["emplacement_pk_start","emplacement_pk_end"]).count()
deplacement_unique = deplacement_unique.reset_index()
deplacement_unique.head()
deplacement_unique[deplacement_unique["emplacement_pk_start"]==deplacement_unique["emplacement_pk_end"]]["start_date"].sum()

We success to import the 5 millions deplacements of Montrealers in 2021. 
Now we can import the stations because we have only the *pk* and not the exact emplacement

In [ ]:
# stations = pd.read_csv('../transfo-data/2021_stations.csv')
stations = pd.read_csv('../data/2021_stations.csv')
stations.loc[stations["latitude"]==-1.0, "latitude"]=45.49277
stations.loc[stations["longitude"]==-1.0, "longitude"]=-73.55641
stations.head()


As you can see, we correct an error found earlier for the coordinates of the Peel/Smith station

We now perform a merge to obtain the geographical locations of each path

In [ ]:
start = deplacement_unique.merge(stations, left_on="emplacement_pk_start", right_on="pk")
start.rename({"latitude":"pickup_latitude", "longitude":"pickup_longitude", "start_date":"nb", "name":"pickup_name"},errors="raise",axis='columns',inplace=True)
start_end = start.merge(stations, left_on="emplacement_pk_end", right_on="pk")
start_end.rename({"latitude":"dropoff_latitude", "longitude":"dropoff_longitude"},errors="raise",axis='columns',inplace=True)
deplacement_final = start_end.drop(columns=["emplacement_pk_end", "emplacement_pk_start","pk_x","pk_y"])
deplacement_final.head()

In [ ]:
deplacement_final["name"]=deplacement_final["pickup_name"]+deplacement_final["name"]
deplacement_final.set_index("name",inplace=True)
deplacement_final.drop("pickup_name",axis=1)
deplacement_final.head()

In [ ]:
# Drop rows where same pickup and dropoff location
index_to_drop=deplacement_final[(deplacement_final["pickup_latitude"]==deplacement_final["dropoff_latitude"]) | (deplacement_final["pickup_longitude"]==deplacement_final["dropoff_longitude"])].index
deplacement_final.drop(index_to_drop,inplace=True)

deplacement_final.to_csv('../data/2021_deplacements.csv')

A file containing all the trips is now saved and can be used in Keglergl or in app.py